# Modeling Walkthrough

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

## Modeling Steps

1. Build a model based on the [Titanic dataset](https://www.kaggle.com/c/titanic/data) that predicts whether a given person survived or not
2. Evaluate the performance of the model
3. Make changes in an attempt to improve the model
4. Demonstrate whether an improvement was made

## The Data

This dataset has the following columns:

| Variable | Definition | Key |
| -------- | ---------- | --- |
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex | |
| Age | Age in years | |
| sibsp | # of siblings / spouses aboard the Titanic | |
| parch | # of parents / children aboard the Titanic | |
| ticket | Ticket number | |
| fare | Passenger fare | |
| cabin | Cabin number | |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

## Initial Data Understanding and Preparation

Open up the file, get everything into `X` features and `y` target variables, divided into train and test

In [ ]:
df = pd.read_csv("titanic.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

Age data is missing for about 1 in 9 rows in our dataset.  For now, let's just exclude it, plus the non-numeric columns, and `PassengerId` which doesn't seem like a real feature, but rather just an artifact of the dataset.

In [ ]:
columns_to_use = ["Survived", "Pclass", "SibSp", "Parch", "Fare"]

In [ ]:
df["Survived"].value_counts()

Not a huge class imbalance, but not evenly-sized categories either

In [ ]:
sns.pairplot(df[columns_to_use])

In [ ]:
df_to_use = df[columns_to_use]
X = df_to_use.drop("Survived", axis=1)
y = df_to_use["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2019)

## Modeling

Let's start with a completely "dummy" model, that will always choose the majority class.

In [ ]:
dummy_model = DummyClassifier(strategy="most_frequent")

Fit the model on our data

In [ ]:
dummy_model.fit(X_train, y_train)

We should expect all predictions to be the same

In [ ]:
# just grabbing the first 50 to save space
dummy_model.predict(X_train)[:50]

## Model Evaluation

In [ ]:
dummy_model.score(X_train, y_train)

So, the mean accuracy is about 60% if we always guess the majority class

In [ ]:
plot_confusion_matrix(dummy_model, X_train, y_train)

A pretty lopsided confusion matrix!

## Modeling, Again

Let's use a logistic regression and compare its performance

In [ ]:
logreg_model = LogisticRegression(random_state=2019)

In [ ]:
logreg_model.fit(X_train, y_train)

Look at the predictions

In [ ]:
logreg_model.predict(X_train)[:50]

Mixture of 1s and 0s this time

## Model Evaluation, Again

In [ ]:
logreg_model.score(X_train, y_train)

So the mean accuracy is 70% if the model is actually taking in information from the features instead of always guessing the majority class

In [ ]:
plot_confusion_matrix(logreg_model, X_train, y_train)

In [ ]:
confusion_matrix(y_train, logreg_model.predict(X_train))

So, in general we are not labeling many of the "not survived" passengers as "survived", but for "survived" passengers we're getting it right only about half of the time

## Data Understanding and Preparation, Again

Maybe there is some useful information in the features we are not using yet.

In [ ]:
df.columns

In [ ]:
df["Name"]

Maybe we could do some parsing to make more sense of this, but that seems complicated

In [ ]:
df["Sex"]

This one is of type "object" but looks potentially one-hot-encode-able

In [ ]:
df["Sex"].value_counts()

In [ ]:
sns.catplot(x="Sex", y="Survived", data=df, kind="bar")

Looks like there is a meaningful difference in survival rates, so let's add it to the model

Only two categories, so we only need a LabelEncoder (no new columns needed, we just need to replace the strings with numbers)

In [ ]:
# re-do the train test split to add another feature
columns_to_use.append("Sex")
df_to_use = df[columns_to_use]
X = df_to_use.drop("Survived", axis=1)
y = df_to_use["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2019)

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
sex_labels = label_encoder.fit_transform(X_train["Sex"])
sex_labels[:50]

In [ ]:
label_encoder.classes_

So, this is telling us that "female" is encoded as 0, "male" is encoded as 1

In [ ]:
X_train["sex_encoded"] = sex_labels
X_train = X_train.drop("Sex", axis=1)

## Modeling and Model Evaluation, Again

Let's see how the logistic regression does, now that this new feature has been added

In [ ]:
second_logreg_model = LogisticRegression(random_state=2019)

In [ ]:
second_logreg_model.fit(X_train, y_train)

In [ ]:
second_logreg_model.score(X_train, y_train)

In [ ]:
plot_confusion_matrix(second_logreg_model, X_train, y_train)

In [ ]:
confusion_matrix(y_train, second_logreg_model.predict(X_train))

So, we have improved the mean accuracy from about 70% to 79%, and actually are doing a better job guessing both the "survived" and "not survived" classes

## Data Understanding and Preparation, Part 3

Anything else we want to do with our dataset?

In [ ]:
X_train.describe()

Looks like Fare is a much bigger number than the others.  Maybe we apply a standard scaler so everything has the same scale

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X_train)

In [ ]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=['Pclass', 'SibSp', 'Parch', 'Fare', 'sex_encoded'])

In [ ]:
X_train.head()

In [ ]:
third_logreg_model = LogisticRegression(random_state=2019)
third_logreg_model.fit(X_train, y_train)
third_logreg_model.score(X_train, y_train)

Didn't break anything, didn't fix anything

## Cross Validation

Let's make sure we are not just overfitting to the training data

In [ ]:
# make a fourth model that is the same as the third one, just not fitted yet
fourth_logreg_model = LogisticRegression(random_state=2019)
cross_val_score(fourth_logreg_model, X_train, y_train)

So, depending on the way the data is sliced, we have somewhere between 74.6% and 81.2% mean accuracy

## Hyperparameter Tuning

Logistic regression has a number of hyperparameters, let's look at the documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

It says that the 'liblinear' solver is not the default, but it is good for small datasets (our data is fewer than 1k rows, which is "small"), so let's try that

In [ ]:
fifth_logreg_model = LogisticRegression(random_state=2019, solver='liblinear')
cross_val_score(fifth_logreg_model, X_train, y_train)

Slightly worse

Let's try a different penalty

In [ ]:
sixth_logreg_model = LogisticRegression(random_state=2019, penalty='none')
cross_val_score(sixth_logreg_model, X_train, y_train)

Doesn't seem to make a big difference.  Doesn't seem like hyperparameter tuning matters much

## Final Evaluation

Now let's use the test data we haven't used at all yet

In [ ]:
columns_to_use

In [ ]:
# read fresh copy from CSV
final_df = pd.read_csv("titanic.csv")

# preprocessing
columns_to_use = ["Survived", "Pclass", "SibSp", "Parch", "Fare", "Sex"]
df_to_use = df[columns_to_use]
X = df_to_use.drop("Survived", axis=1)
y = df_to_use["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2019)

In [ ]:
def apply_preprocessing(features):
    label_encoder = LabelEncoder()
    sex_labels = label_encoder.fit_transform(features["Sex"])
    features["sex_encoded"] = sex_labels
    features = features.drop("Sex", axis=1)

    scaler = StandardScaler()
    features = pd.DataFrame(scaler.fit_transform(features), columns=['Pclass', 'SibSp', 'Parch', 'Fare', 'sex_encoded'])
    return features

In [ ]:
X_train = apply_preprocessing(X_train)
X_test = apply_preprocessing(X_test)

In [ ]:
final_model = LogisticRegression(random_state=2019)
final_model.fit(X_train, y_train)

In [ ]:
final_model.score(X_test, y_test)